Title: Supervised Learning 

Task 1: Identify input-output pairs in a dataset of housing prices.

In [ ]:
import pandas as pd

# Create a sample housing price dataset
data = {
    'House Size': [1500, 2000, 1800, 2200, 2500],
    'Number of Bedrooms': [3, 4, 3, 4, 5],
    'Number of Bathrooms': [2, 3, 2, 3, 4],
    'Location': ['Zip Code 12345', 'Zip Code 67890', 'Zip Code 34567', 'Zip Code 90123', 'Zip Code 45678'],
    'Age of the House': [10, 5, 15, 8, 12],
    'House Price': [500000, 700000, 550000, 800000, 900000]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Identify input-output pairs
input_features = df[['House Size', 'Number of Bedrooms', 'Number of Bathrooms', 'Age of the House']]
output = df['House Price']

# Print the input features and output
print("Input Features:")
print(input_features)
print("\nOutput:")
print(output)


Task 2: Match labeled images to their correct classifications.

In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

# Define constants
TRAIN_DIR = 'path/to/train/directory'
TEST_DIR = 'path/to/test/directory'
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 32

# Data augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load the training and testing data
train_generator = train_datagen.flow_from_directory(TRAIN_DIR,
                                                    target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(TEST_DIR,
                                                  target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                  batch_size=BATCH_SIZE,
                                                  class_mode='categorical')

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

# Freeze the base layers
base_model.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=test_generator)

# Plot the training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()

# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy:.2f}')

# Fine-tune the model
base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_generator,
                    epochs=10,
                    validation_data=test_generator)

# Evaluate the fine-tuned model
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy after fine-tuning: {accuracy:.2f}')



Task 3: List potential inputs and outputs for predicting stock prices.

In [ ]:


# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import yfinance as yf

# Download historical stock data
stock_data = yf.download('AAPL', start='2010-01-01', end='2022-02-26')

# Prepare the data
stock_data['Date'] = pd.to_datetime(stock_data.index)
stock_data['Day'] = np.arange(len(stock_data))
stock_data.set_index('Day', inplace=True)

# Define the features and target
X = stock_data[['Open', 'High', 'Low']]
y = stock_data['Close']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')

# Plot the predicted prices
plt.plot(y_test.values, label='Actual Price')
plt.plot(y_pred, label='Predicted Price')
plt.legend()
plt.show()

# Use the model to predict future prices
future_data = pd.DataFrame({'Open': [100, 110, 120], 'High': [110, 120, 130], 'Low': [90, 100, 110]})
future_price = model.predict(future_data)
print(f'Predicted Future Price: {future_price}')

